In [154]:
from collections import Counter, namedtuple
import numpy as np
from typing import Tuple, List, Optional, Sequence

In [2]:
ar = np.sqrt(np.array([.5, 1, 2]))
scale = np.array([8, 16, 32])

In [3]:
x_out, y_out = (32, 32)
y, x = np.meshgrid(np.arange(x_out), np.arange(y_out))
base_h, base_w = 32, 32

In [4]:
x = x.reshape(-1, ).repeat(9)
y = y.reshape(-1, ).repeat(9)
x = x * base_w + base_w / 2
y = y * base_h + base_h / 2

In [5]:
ar = np.tile(ar, x_out * y_out).repeat(3)
scale = np.tile(scale, x_out * y_out * 3)

In [6]:
hs = np.round(base_h * scale * ar).astype(int)
ws = np.round(base_w * scale / ar).astype(int)

In [55]:
boxes = np.vstack([x, y, ws, hs]).T

In [56]:
def generate_anchors(
    stride: int = 16,
    scales: List[int] = [8, 16, 32],
    ratios: List[int] = [1, 0.5, 2],
    base_dist: Optional[int] = None,
) -> np.ndarray:
    """computes anchor boxes for a given stride, scales and aspect ratios.
        Number of anchors = stride ** 2 * scales * aspect_ratio

    Keyword Arguments:
        stride {int} -- the subsampling ratio of the output feature map (default: {16})
        scales {List[int]} -- list of scales (default: {[8, 16, 32]})
        ratios {List[int]} -- list of aspect ratios (default: {[1, 0.5, 2]})

    Returns:
        [np.ndarray] -- shape: `(stride * stride * scales * ratios, 4)`
    """
    base_dist = base_dist if base_dist is not None else stride

    x_out, y_out = stride, stride
    base_h, base_w = base_dist, base_dist
    y, x = np.meshgrid(np.arange(x_out), np.arange(y_out))

    x = x.reshape(-1,).repeat(len(scales) * len(ratios))
    y = y.reshape(-1,).repeat(len(scales) * len(ratios))
    x = x * base_w + base_w / 2
    y = y * base_h + base_h / 2

    ratios = np.tile(np.array(ratios), x_out * y_out).repeat(3)
    scales = np.tile(np.array(scales), x_out * y_out * 3)

    hs = np.round(base_h * scales * ratios).astype(int)
    ws = np.round(base_w * scales / ratios).astype(int)

    return np.vstack([x, y, ws, hs]).T

def hw_to_minmax(bbox: np.ndarray, max_dim: Tuple[int, int]) -> np.ndarray:
    """given a set of bounding (anchor) boxes in the format `(x_center, y_center, width, height)`,
        transforms them to `(x_min, y_min, x_max, y_max)` format
    """
    boxes = np.zeros_like(bbox, dtype=np.int32)
    max_x, max_y = max_dim

    boxes[:, 0], boxes[:, 2] = (
        np.maximum(bbox[:, 0] - bbox[:, 2] / 2, 0).astype(int),
        np.minimum(bbox[:, 0] + bbox[:, 2] / 2, max_x).astype(int),
    )

    boxes[:, 1], boxes[:, 3] = (
        np.maximum(bbox[:, 1] - bbox[:, 3] / 2, 0).astype(int),
        np.minimum(bbox[:, 1] + bbox[:, 3] / 2, max_y).astype(int),
    )
    return boxes

In [204]:
def get_area(bbox):
    return (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

def iou(box1, box2, eps = 1e-7):
    xmin = max(box1[0], box2[0])
    ymin = max(box1[1], box2[1])
    xmax = min(box1[2], box2[2])
    ymax = min(box1[3], box2[3])
    if (xmin >= xmax) or (ymin >= ymax):
        return 0
    intersection = (xmax - xmin) * (ymax - ymin)
    union = get_area(box1) + get_area(box2) - intersection
    return intersection / (union + eps)

def iou_vectorized(anchors, bbox, eps = 1e-7):
    xmin = np.maximum(anchors[:, 0], bbox[0])
    ymin = np.maximum(anchors[:, 1], bbox[1])
    xmax = np.minimum(anchors[:, 2], bbox[2])
    ymax = np.minimum(anchors[:, 3], bbox[3])
    
    intersection = np.maximum((xmax - xmin) * (ymax - ymin), 0.)
    intersection[np.where((xmin >= xmax) | (ymin >= ymax))[0]] = 0.
    union = (anchors[:, 2] - anchors[:, 0]) * (anchors[:, 3] - anchors[:, 1]) + get_area(bbox) - intersection
    return intersection / (union + 1e-7)

def make_labels(anchors, boxes, ignore = -1, img_dim = 512, n_classes = 1):
    minmax_anchors = hw_to_minmax(anchors, max_dim=(img_dim, img_dim))
    cls_gt = np.repeat(ignore, len(anchors) * n_classes).reshape(len(anchors), n_classes)
    #cls_gt = []
    for box in boxes: 
        class_id = box.class_id
        bbox = list(box.bbox)
        ious = iou_vectorized(minmax_anchors, bbox)
        cls_gt[np.where(ious > 0.7)[0], class_id - 1] = 1
        
        if (cls_gt == ignore).all():
            cls_gt[np.argmax(ious), class_id - 1] = 1
    
        cls_gt[np.where(ious < 0.3)[0], class_id - 1] = 0

    return cls_gt

In [180]:
Bbox = namedtuple('Bbox', 'xmin ymin xmax ymax')
Label = namedtuple('Label', ['class_id', 'bbox'])

In [181]:
lbl1 = Label(1, Bbox(0, 0, 140, 140))
lbl2 = Label(1, Bbox(20, 60, 200, 350))
lbl = [lbl1, lbl2]

In [187]:
anchors = generate_anchors(stride=32)

In [205]:
make_labels(anchors, lbl, img_dim=1024, ignore=255)

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [167]:
l = np.array([2, 3, 9, 1, 7])

In [169]:
a[np.where(l % 3 == 0), 1] = -100

In [170]:
a

array([[   0,    1,    2,    3,    4],
       [   5, -100,    7,    8,    9],
       [  10, -100,   12,   13,   14],
       [  15,   16,   17,   18,   19],
       [  20,   21,   22,   23,   24]])